In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import glob
import argparse
import random
import numpy as np
import lmdb
import cv2
import pickle
import caffe
from caffe.proto import caffe_pb2
from time import time
from sklearn.model_selection import train_test_split

## Directory parser for file .py

In [2]:
args = '/home/helpthx/TCC-1/TCC-1-UnB//home/helpthx/Documents/TCC-1-dataset/dataset-split'

## Default imagem size for resnet architecture 

In [3]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

In [4]:
def transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT, equalize=False):
  """Function that resize an image and equalize it if necessary."""
  if equalize:
    # Histogram Equalization
    img[:, :, 0] = cv2.equalizeHist(img[:, :, 0])
    img[:, :, 1] = cv2.equalizeHist(img[:, :, 1])
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])
    
  # Image Resizing
  img = cv2.resize(img, (img_width, img_height), interpolation=cv2.INTER_CUBIC)
  
  return img

In [5]:
def make_datum(img, label):
  # Image is numpy.ndarray format. BGR instead of RGB
  return caffe_pb2.Datum(channels=3,
                         width=IMAGE_WIDTH,
                         height=IMAGE_HEIGHT,
                         label=label,
                         data=np.rollaxis(img, 2).tostring())

## Creating train and validation datasets with lmdb lib

In [13]:
path = '/home/helpthx/Documents/TCC-1-dataset/dataset-split'
parent_path = os.path.sep.join(path.split(os.path.sep)[:-1])
sibling_path = path.split(os.path.sep)[-1] + '_lmdb'
sibling_path = os.path.sep.join([parent_path, sibling_path])
train_lmdb = os.path.sep.join([sibling_path, 'train'])
validation_lmdb = os.path.sep.join([sibling_path, 'validation'])

In [14]:
print('train lmdb path: ', train_lmdb)
print('validation path: ', validation_lmdb)
print('sibling path: ', sibling_path)
print('parent path: ', parent_path)

train lmdb path:  /home/helpthx/Documents/TCC-1-dataset/dataset-split_lmdb/train
validation path:  /home/helpthx/Documents/TCC-1-dataset/dataset-split_lmdb/validation
sibling path:  /home/helpthx/Documents/TCC-1-dataset/dataset-split_lmdb
parent path:  /home/helpthx/Documents/TCC-1-dataset


In [15]:
if not os.path.exists(sibling_path):
    os.makedirs(sibling_path)

In [17]:
dataset = []
for r, dirs, files in os.walk(path):
    if len(dirs) > 0:
        labels = dirs
        continue # use only leaf folders
    files_full_path = ['{}/{}'.format(r, f) for f in files]
    directory_name = r.split(os.path.sep)[-1]
    dataset.append((files_full_path, directory_name))

['hemangioma_original_4174.jpg_cac11b21-a1db-458d-934b-411a4fd136f1.jpg',
 'hemangioma_original_4142.jpg_57d1ccf8-4c8e-440b-8ae3-f148eddcda5d.jpg',
 'hemangioma_original_4232.jpg_97db0775-7a54-49f9-8289-68c82604ae26.jpg',
 'hemangioma_original_4102.jpg_db1e9aee-fd38-4554-b04f-1f2f36bb7a21.jpg',
 'hemangioma_original_4227.jpg_60f302d2-8bd5-4463-b0a5-a878ce116558.jpg',
 'hemangioma_original_4213.jpg_dcaf7780-1b57-47df-8551-eb652ee65f9d.jpg',
 'hemangioma_original_4238.jpg_8e4349d5-0070-439d-9b5b-127c3a6645c9.jpg',
 'hemangioma_original_4244.jpg_66631ea7-a840-4eb1-b60a-1cf0aa67a4aa.jpg',
 'hemangioma_original_4141.jpg_91cdf890-11c8-40ca-b248-dec4d4824817.jpg',
 'hemangioma_original_4173.jpg_67ab1589-97be-4ae3-b6e4-45c6c2b500b4.jpg',
 'hemangioma_original_4090.jpg_9caf0272-3c46-4fc1-a2d6-3f3bfed35976.jpg',
 'hemangioma_original_4200.jpg_ffc95d7d-129d-45f2-a6b8-966133b16908.jpg',
 'hemangioma_original_4201.jpg_09c9625d-9908-4193-86bd-39869701bd7c.jpg',
 'hemangioma_original_4116.jpg_61c55a6

In [18]:
label_dict = [(l, i) for i, l in enumerate(labels)]
label_dict = dict(label_dict)
label_dict

{'dermatofibroma': 0,
 'malignant-melanoma': 1,
 'squamous-cell-carcinoma': 2,
 'pyogenic-granuloma': 3,
 'lentigo': 4,
 'actinic-keratosis': 5,
 'intraepithelial-carcinoma': 6,
 'melanocytic-nevus': 7,
 'seborrheic-keratosis': 8,
 'wart': 9,
 'basal-cell-carcinoma': 10,
 'hemangioma': 11}

'''
Save dictionary in the form of:

label_dict = { \
'basalcellcarcinoma': 0, \
'lentigo': 1, \
'malignantmelanoma': 2, \
'pigmentednevus': 3, \
'seborrheickeratosis': 4, \
'wart': 5, \
... \
}
'''